In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
import qdrant_client
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings



In [15]:
# R

In [16]:
import os
# os.getenv("OPENAI_API_KEY")

In [17]:
# OpenAI Embedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(embed_batch_size=10, model="text-embedding-3-small")
Settings.embed_model = embed_model #later should not use global varr


In [18]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

In [19]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

import qdrant_client

# Create qdrant client
client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

In [20]:
import json

# load json
fileName = "data/20250612.json"

with open(fileName, "r", encoding="utf-8") as file:
    data = json.load(file)

In [21]:
from llama_index.core import Document

documents = []
for dp in data:
    documents.append(Document(
        text = dp["label"],
        doc_id = dp["id"],
        metadata = dp,
    ))
    
# print(documents[0].metadata)

In [28]:
# create vector store index with batching to avoid timeout
vector_store = QdrantVectorStore(client=client, collection_name="linh-test")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Batch size for ingestion (adjust as needed)
batch_size = 200

for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    index = VectorStoreIndex.from_documents(
        batch_docs,
        storage_context=storage_context,
    )

ResponseHandlingException: timed out

In [ ]:
# Run validation
import pandas

validation_data = pandas.read_excel('./data/Report_MappingCSYT_20250507.xlsx', sheet_name="Detailed", usecols=['Tên CSYT trên chứng từ', 'Tên CSYT mapping'])
print(validation_data)


                                Tên CSYT trên chứng từ  \
0                                     Nha khoa Parkway   
1                                 Công ty TNHH Parkway   
2                 Đại học Y Dược Thành phố Hồ Chí Minh   
3                       Đại học Y Dược TP. Hồ Chí Minh   
4    CÔNG TY TNHH MỘT THÀNH VIÊN DƯỢC PHẨM VÀ TRANG...   
..                                                 ...   
669                  bệnh viện đa khoa tâm trí đà nẵng   
670                  bệnh viện đa khoa tâm trí đà nẵng   
671  CÔNG TY TNHH MỘT THÀNH VIÊN BỆNH VIỆN ĐA KHOA ...   
672                  bệnh viện đa khoa tâm trí đà nẵng   
673                   tam tri da nang general hospital   

                                      Tên CSYT mapping  
0                                     Nha khoa Parkway  
1                          Công ty TNHH Medic Thành An  
2    Bệnh viện Đại học Y dược Tp.Hồ Chí Minh (Cơ sở 3)  
3    Bệnh viện Đại học Y dược Tp.Hồ Chí Minh (Cơ sở 3)  
4                 

In [ ]:
data_dict = validation_data.to_dict(orient='records')
print('Excel Sheet to Dict:', data_dict)

['Nha khoa Parkway', 'Công ty TNHH Parkway', 'Đại học Y Dược Thành phố Hồ Chí Minh', 'Đại học Y Dược TP. Hồ Chí Minh', 'CÔNG TY TNHH MỘT THÀNH VIÊN DƯỢC PHẨM VÀ TRANG THIẾT BỊ Y TẾ PHƯƠNG CHÂU', 'BENH VIEN QUOC TE PHUONG CHAU', 'BỆNH VIỆN QUỐC TẾ PHƯƠNG CHÂU', 'CHI NHÁNH CÔNG TY CỔ PHẦN ĐẦU TƯ VÀ THƯƠNG MẠI PHƯƠNG CHÂU BỆNH VIỆN QUỐC TẾ PHƯƠNG CHÂU', 'BỆNH VIỆN HOÀN MỸ THỦ ĐỨC', 'BỆNH VIỆN ĐA KHOA QUỐC TẾ HOÀN MỸ THỦ ĐỨC', 'CÔNG TY CỔ PHẦN BỆNH VIỆN ĐA KHOA QUỐC TẾ HOÀN MỸ THỦ ĐỨC', 'BỆNH VIỆN TỪ DŨ', 'BỆNH VIỆN TỪ DŨ Khu 191', 'BỆNH VIỆN TỪ DŨ', 'BỆNH VIỆN TỪ DŨ Nhà thuốc 191 NTMK', 'SỞ Y TẾ HÀ NỘI BỆNH VIỆN PHỤ SẢN HÀ NỘI', 'BỆNH VIỆN PHỤ SẢN HÀ NỘI', 'BỆNH VIỆN PHỤ SẢN HÀ NỘI', 'BỆNH VIỆN PHỤ SẢN HÀ NỘI Nhà thuốc bệnh viện 2', 'Đại học Y Dược Thành phố Hồ Chí Minh Nhà thuốc 215', 'Đại học Y Dược Thành phố Hồ Chí Minh', 'BỆNH VIỆN E', 'BỆNH VIỆN E', 'BỆNH VIỆN CHÂM CỨU TW', 'BỆNH VIỆN CHÂM CỨU TRUNG ƯƠNG', 'BỆNH VIỆN CHÂM CỨU TRUNG ƯƠNG', 'BỆNH VIỆN ĐA KHOA HÒA HẢO-MEDIC CẦN THƠ', 'C

In [27]:
# Query
# query_engine = index.as_query_engine()


# run through eevry obj in the dict --> compare respective data in 'Tên CSYT trên chứng từ' and 'Tên CSYT mapping'
for dp in data_dict:
    # for k in dp.keys():
    #     print(k,":", dp[k])
    # print(dp[0], dp[1])
    cs_chungtu = dp['Tên CSYT trên chứng từ']
    cs_mapping = dp['Tên CSYT mapping']
    
    query_engine = index.as_query_engine()
    response = query_engine.query(cs_chungtu)
    

NameError: name 'data_dict' is not defined